# Data Crawling

Sadly, there are times wherein data will not be handed on a silver platter there are times wherein you have to gather data yourselves. Gaining exposure through various data collection methods will be a vital skill to have in order to thrive as a budding data scientist. 

In this notebook, we will be gathering data from through an Application Programming Interface(API). An API is an interface that allows programs to communicate with each other. A wide use of APIs can be found on online banking transactions. Usually, banks would expose APIs that allow developers cerain access to the bank's system (ex. money transfer, balance checking). In turn, developers will use these APIs to build the functionalities of their applications. In our case we use APIs to extract data to do our analysis and build our models. 

<img src="https://miro.medium.com/max/2400/1*vhoE-Yw2HgrlScZmR_L1zA.gif"/>Sourced from <a href="https://medium.com/@JanisGraubins/open-banking-api-explained-in-3-gifs-b806f14ca2ca"> Grabuis Jannis</a></img>

In [1]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

To familiarize ourselves with the use APIs we will be extracting data from <a href="reddit.com">reddit</a>. Reddit is a media aggregation website that contains multiple communities. You guys can imagine it as a "forum-esque" website where users can discuss about anything under the sun. The tool that we will be using to get reddit data is <a href="https://pushshift.io/"> pushift</a>. Pushshift, is an open source big data storage platform that copies submissions on reddit. The developer of this project has exposed an API which allows users to search for comments, aggregate data, query data for a specific range and many <a href="https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/">more</a>. One limitation of the tool is that it saves the metadata of the submissions the time it was posted on reddit therefore scores, upvotes and downvotes are not updated.

# Let's Start!

To access pushshift we will be using the `requests` library of Python. This library allows us to do HTTP requests on the API. Basically, this library allows us to communicate with the API. 

There are 3 main entry to points for the API these are: <br>
`/reddit/comment/search` <br>
`/reddit/submission/search` <br>
`/reddit/subreddit/search` <br>

This means you can search based on texts coming from comments, submissions or subreddit. To filter your query the API offers a `param ` attribute. There are various parameters which you can find <a href="https://github.com/pushshift/api">here</a>. For this tutorial we will only be using 7. <br>

`after` - filter results to those made after a given date <br>
`before`- filter results to those made before a given date <br>
`sort_type` - sort by a specific attribute default value is "created_utc" other options are  ("score", "num_comments", "created_utc") <br>
`sort` - sort results in specific order ("asc" or "desc") <br>
`subreddit` - name of subreddit <br>
`size` - number of results to return <br>
`fields` - specifcy which fields to return <br>

## Sample API Query 
The query below gets submissions from DLSU's <a href="https://www.reddit.com/r/dlsu/">reddit</a> community from April 1, 2020 to April 30, 2020 sorted in descending order based on score with only 20 results. We also indicate what fields the API should return. 

If you notice there is something weird about the date we placed in the `after` and `before` parameters. The API uses an epoch timestamp. This is the timestamp used by Unix. For us to interface with the API properly we can use this <a href="https://www.unixtimeconverter.io/">time converter</a>. Just type in the date that you want to be converted to Unix. 

You may notice that after calling the requests library we use the `.json` method. This converts the results of the request into a JSON object. JSON is a widely used file format usually used to communicate between applications. In our case Reddit API and this notebook.

In [2]:
URL = "https://api.pushshift.io/reddit/submission/search/"  #query submissions
PARAMS = {
    'after': 1483228800, #get dates after March 31, 2020
    'before': 1588291200, #get dates before May 1, 2020
    'sort_type': 'score', # sort by score
    'sort': 'desc', # sort in descending order
    'subreddit': 'dlsu', # do a search on DLSU subreddit
    'size': 20, # give only 20 search results
#     'fields': ["id","title","selftext","score","num_comments","created_utc"] #return only the following fields
}

#use the requests library to query pushshift api
r = requests.get(url = URL, params=PARAMS)
#parse returned data to a json object
r.json()

{'data': [{'author': 'AlphaPotatoe',
   'author_flair_css_class': None,
   'author_flair_richtext': [],
   'author_flair_text': None,
   'author_flair_type': 'text',
   'author_fullname': 't2_2yhhhl7e',
   'author_patreon_flair': False,
   'can_mod_post': False,
   'contest_mode': False,
   'created_utc': 1551253297,
   'domain': 'i.redd.it',
   'full_link': 'https://www.reddit.com/r/dlsu/comments/avasux/make_an_infographic_of_la_salle_some_teacher/',
   'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0},
   'id': 'avasux',
   'is_crosspostable': True,
   'is_meta': False,
   'is_original_content': False,
   'is_reddit_media_domain': True,
   'is_robot_indexable': True,
   'is_self': False,
   'is_video': False,
   'link_flair_background_color': '',
   'link_flair_richtext': [],
   'link_flair_text_color': 'dark',
   'link_flair_type': 'text',
   'locked': False,
   'media_only': False,
   'no_follow': True,
   'num_comments': 1,
   'num_crossposts': 0,
   'over_18': False,
   'permalink

# Scaling the API Query 

What if we want to scale the API and be able to get all reddit posts from April 1, 2020 to April 30, 2020? One way to do this would be to call the API for each day from Apirl 1 to April 30. For each day we set the size of the results to 500. The reason we are limiting it to 500 is the API only accepts values <= 500 for the size parameter. 

Note: <br>
We can see a limitation in this approach, what if a reddit page gets more than 500 posts a day. This a reality that you may encounter when dealing with APIs there will be limitations in how much you can query or extract. In the case of reddit, pushshift has provided a data dump through this <a href="https://files.pushshift.io/reddit/">link</a>.

In [25]:
def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-04-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-04-05", "%Y-%m-%d")

#Create a range of dates to iterate 
#Note: Periods here represents the number of days it will create from the start date 
#We also do a +2 since it will only generate up to April 29. We inlcude May 1 
#since we want to get data from the last day which is April 30 to May 1 
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#prepare the parameters needed to call the API
sort_type="score"
sort="desc"
fields=["id","title","selftext","score","num_comments","created_utc"]
subreddit = 'dlsu'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)

Doing 2020-04-01 to 2020-04-02
=====Done
Doing 2020-04-02 to 2020-04-03
=====Done
Doing 2020-04-03 to 2020-04-04
=====Done
Doing 2020-04-04 to 2020-04-05
=====Done
Doing 2020-04-05 to 2020-04-06
=====Done


The results variable will be a list of lists and within a list is a dictionary object. Now for us to properly save this to a dataframe we would need to flatten the list and create a list of dictionary objects instead. 

In [27]:
results

[[{'created_utc': 1585734263,
   'id': 'fswq91',
   'num_comments': 4,
   'score': 1,
   'selftext': '🤪',
   'title': 'Do you believe that dlsu is number two univ?'},
  {'created_utc': 1585712182,
   'id': 'fsrv3e',
   'num_comments': 0,
   'score': 1,
   'selftext': "I am currently having some problems with course crediting. I applied to graduate a few weeks ago and also sent the registrar ([registrar@dlsu.edu.ph](mailto:registrar@dlsu.edu.ph)) an email regarding my course crediting form, which I submitted early January (sobrang tagal na), but still has not been credited. I saw some help desk announcements today that was sent by the registrar's email, which means they should've seen my email right? Do I send a follow-up email regarding my course crediting, since the ATG is until April 4 only? Or should I wait for them to reply?",
   'title': 'Application to graduate problems. When will the registrar resume operations?'},
  {'created_utc': 1585719592,
   'id': 'fstmev',
   'num_comment

## Converting data into a table

Given that the JSON representation is a bit complex, additional processing would need to be done to be able to reogranize or restructure the data into a "flat" list of dictionary objects.

Once we have the flat list, we can easily pass this to `pandas` which will make our data easier to analyze.

In [26]:
flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty lists is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head())
df.to_csv("reddit_dlsu.csv")

,created_utc,id,num_comments,score,selftext,title
0,1585734263,fswq91,4,1,🤪,Do you believe that dlsu is number two univ?
1,1585712182,fsrv3e,0,1,I am currently having some problems with cours...,Application to graduate problems. When will th...
2,1585719592,fstmev,3,1,,"Need Respondents from Mangement, I.T. and Econ..."
3,1585765380,ft5kcl,10,1,Hi I’m an incoming dlsu college student this s...,What’s the best dlsu course to take to become ...
4,1585768156,ft6gc1,0,1,Does anyone know what first years in game deve...,Flowchart for IET Game dev
